In [ ]:
import numpy as np
import json
import pandas as pd
from pandas.io.json import json_normalize

In [ ]:
import glob
import os

path = 'C://Users/Tom/Documents/Kickstarter/JSONs' 
list_files = os.listdir(path)
files = []

#files = [i for i in files if i.endswith('.json')]
for file in list_files:
    file = path + '/' + file
    files.append(file)

In [ ]:
### NOTE! This code takes about 4 hours to run! ###

master = []

# For file in list:
for file in files:
# Read file using pd.read_json
    #json.load(open("path"))
    df = pd.read_json(file, convert_dates=True, encoding='utf-8', lines=True)
# Normalize the data using json_normalize
    projects = json_normalize(df['data'])
# Filter unnecessary fields
    new_projects = projects[['state', 'goal', 'usd_pledged', 'static_usd_rate', 'backers_count', 'blurb', 'category.name', 'category.slug', 'category.id', 'launched_at', 'deadline', 'id', 'location.country', 'location.state', 'location.type', 'spotlight', 'staff_pick']]
# Append files to master dataset
    master.append(new_projects)
# Concatenate files together to build one massive dataset
mastercsv = pd.concat(master, axis='rows')


In [ ]:
mastercsv = pd.read_csv('masterglob.csv')
pd.DataFrame(mastercsv)
mastercsv.reset_index(inplace=True, drop=True)


In [ ]:
# Feature Engineering and Cleaning Columns
import datetime
mastercsv['launched_at'] = mastercsv['launched_at'].apply(datetime.datetime.utcfromtimestamp)
mastercsv['deadline'] = mastercsv['deadline'].apply(datetime.datetime.utcfromtimestamp)

In [ ]:
# Changing usd_pledged into converted_USD
mastercsv['converted_USD'] = mastercsv['usd_pledged'] / mastercsv['static_usd_rate']

In [ ]:
mastercsv['days_to_deadline'] = mastercsv['deadline'] - mastercsv['launched_at']

In [ ]:
# Replacing null values
mastercsv['blurb'] = mastercsv['blurb'].replace(np.nan, '--')
mastercsv['location.country'] = mastercsv['location.country'].replace(np.nan, '--')
mastercsv['location.state'] = mastercsv['location.country'].replace(np.nan, '--')
mastercsv['location.type'] = mastercsv['location.type'].replace(np.nan, '--')

In [ ]:
# Check if blurb is zero
mastercsv[mastercsv['blurb'].isnull()]
mastercsv['blurb'] = mastercsv['blurb'].replace(np.nan, '--')

In [ ]:
mastercsv[mastercsv['location.country'].isnull()]
mastercsv['location.country'] = mastercsv['location.country'].replace(np.nan, '--')

In [ ]:
mastercsv[mastercsv['location.state'].isnull()]
mastercsv['location.state'] = mastercsv['location.country'].replace(np.nan, '--')

In [ ]:
# Splits category slug to contain just parent category
mastercsv['category.slug'] = mastercsv['category.slug'].apply(lambda x: x.split("/")[0] )

In [ ]:
mastercsv.drop_duplicates() #Many projects appeared on multiple scrapes each month

In [ ]:
# Save final version as CSV file:
mastercsv.to_csv('clean.csv')